# Preprocessing of the data from Vinod's MongoDB database

In [155]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm, tqdm_notebook

# Load the data

In [2]:
PATH2 = 'data/vinod/'
f_power_labels = f"{PATH2}Columbia_Enron_DominanceTuples.csv"
f_gender_labels = f"{PATH2}Columbia_Enron_FirstName_Gender_Type.csv"
f_vinod_corpus = f"{PATH2}cleaned_v1_mongo.pkl"

In [3]:
# Load the data
power_labels = pd.read_csv(f_power_labels)
gender_labels = pd.read_csv(f_gender_labels)

# Round-about loading, possibly because it was saved in python2
with open(f_vinod_corpus, 'rb') as f:
    data = pickle.load(f, encoding='latin1')

# Clean the data

In [90]:
# Clean the gender labels
gender_labels = gender_labels[gender_labels['FirstName'] != 'None']
gender_labels.reset_index(drop=True, inplace=True)

# Build dicts for easy dataset merging
gender_dict = defaultdict(lambda: 'I')
employee_dict = defaultdict(lambda: 'unk')
ambiguity_dict = defaultdict(lambda: 'NotFound')
for _, (uid, _, ambig, gender, employee_type) in gender_labels.iterrows():
    gender_dict[uid] = gender
    employee_dict[uid] = employee_type
    try:
        ambig = round(ambig, 3)
    except TypeError:
        pass
    ambiguity_dict[uid] = ambig

In [104]:
# Build a power dict for easy look-up
power_dict = defaultdict(lambda: 'na')

for _, (boss, sub, imd) in power_labels.iterrows():
    power_dict[(boss, sub)] = imd

In [10]:
# Clean out the main messages    
data.dropna(inplace=True)

# Drop ones which lack a from_id
msk = data['from_id'].apply(lambda v: type(v) is int)
data = data[msk]

# Merge the labels into one dataset

In [ ]:
# Make big csv
# Make slimmed csv (main message, power labels, message type) with key to decode int labels

In [91]:
# Add from_id data
data.loc[:, 'from_gender'] = [gender_dict[uid] for uid in data['from_id']]
data.loc[:, 'from_ambig'] = [ambiguity_dict[uid] for uid in data['from_id']]
data.loc[:, 'from_employee_type'] = [employee_dict[uid] for uid in data['from_id']]

# Drop out unknown employee types
data = data[data['from_employee_type'] != 'unk']

In [92]:
# Add to_ids data
data.loc[:, 'to_gender'] = [[gender_dict[uid] for uid in row['to_ids']] for _, row in data.iterrows()]
data.loc[:, 'to_ambig'] = [[ambiguity_dict[uid] for uid in row['to_ids']] for _, row in data.iterrows()]
data.loc[:, 'to_employee_type'] = [[employee_dict[uid] for uid in row['to_ids']] for _, row in data.iterrows()]

In [148]:
# Add cc_ids data
data.loc[:, 'cc_gender'] = [[gender_dict[uid] for uid in row['cc']] for _, row in data.iterrows()]
data.loc[:, 'cc_ambig'] = [[ambiguity_dict[uid] for uid in row['cc']] for _, row in data.iterrows()]
data.loc[:, 'cc_employee_type'] = [[employee_dict[uid] for uid in row['cc']] for _, row in data.iterrows()]

In [ ]:
# Add all power relations

In [210]:
def get_power_relations(from_id, to_ids):
    power_relation_exists = []
    power_relation_imd = []
    for i, to_id in enumerate(to_ids):
        tup, imd = get_power_relation(from_id, to_id)
        if tup != 'na':
            power_relation_exists.append(tup)
            power_relation_imd.append(imd)
    
    return power_relation_exists, power_relation_imd

def get_power_relation(from_id, to_id):
    tup = (from_id, to_id)
    power = power_dict[tup]
    if power == 'na':
        tup = (to_id, from_id)
        power = power_dict[tup]
        if power == 'na':
            tup = 'na'
    return tup, power

In [211]:
# Extract power relations for recipients
power_relations = []
power_imds = []

for i, row in tqdm_notebook(data.iterrows()):
    from_id = row['from_id']
    to_ids = row['to_ids']
    
    power, imds = get_power_relations(from_id, to_ids)
    power_relations.append(power)
    power_imds.append(imds)

data['to_power_rels'] = power_relations
data['to_power_imds'] = power_imds

A Jupyter Widget

In [212]:
# Extract power relations for cc'd ids
power_relations = []
power_imds = []

for i, row in tqdm_notebook(data.iterrows()):
    from_id = row['from_id']
    to_ids = row['cc']
    
    power, imds = get_power_relations(from_id, to_ids)
    power_relations.append(power)
    power_imds.append(imds)

data['cc_power_rels'] = power_relations
data['cc_power_imds'] = power_imds

A Jupyter Widget

# Save out merged data

In [213]:
f_out_large = f"{PATH2}full_data.pkl"
f_out_small = f"{PATH2}full_data_small.pkl"

f_out_power = f"{PATH2}power_relations_only.pkl"


In [214]:
with open(f_out_large, 'wb') as f:
    pickle.dump(data, f)

In [215]:
# Slightly smaller version of the data
data_small = data[['main_text', 'message_type', 'subject',
                   'from_id', 'to_ids', 'cc',
                   'from_gender', 'to_gender', 'cc_gender',
                   'from_employee_type', 'to_employee_type', 'cc_employee_type',
                   'to_power_rels', 'cc_power_rels',
                   'to_power_imds', 'cc_power_imds']]

with open(f_out_small, 'wb') as f:
    pickle.dump(data_small, f)

In [216]:
# Power relations only
msk1 = data['to_power_rels'].apply(lambda p: len(p) > 0)
msk2 = data['cc_power_rels'].apply(lambda p: len(p) > 0)
msk = msk1 | msk2
data_power_only = data[msk]

with open(f_out_power, 'wb') as f:
    pickle.dump(data_power_only, f)

In [217]:
data_power_only

,all_text,cc,from_id,from_name,main_text,message_type,subject,to_ids,to_names,from_gender,...,to_gender,to_ambig,to_employee_type,cc_gender,cc_ambig,cc_employee_type,to_power_rels,to_power_imds,cc_power_rels,cc_power_imds
uid,,,,,,,,,,,,,,,,,,,,,
204,"[{'content': 'Welcome to UBS Warburg Energy, L...",[],18656,Christopher F Calger,"Welcome to UBS Warburg Energy, LLC\n\nAll UBS ...",INITIAL,UBSW ENERGY KICK-OFF CELEBRATION,"[567, 52125, 111470, 117105, 18656, 3786, 5348...","[Susan Amador, John Anderson, Tim Belden, Vict...",M,...,"[F, M, M, M, M, F, F, M, F, F, M, F, M, M, F, ...","[0.439794775, 0.728227303, 0.48073235, 1.21449...","[NonCore, NonCore, NonCore, NonCore, NonCore, ...",[],[],[],"[(18656, 2149)]",[1],[],[]
228,[{'content': 'Preparations are continuing for ...,"[28701, 1939, 18989, 79985, 53860, 14758]",1786,Bob Shults,Preparations are continuing for our simulation...,INITIAL,Online Trading Simulation Feb 7th,"[32825, 32827, 14326]","[DL-NETCO Gas Traders, DL-NETCO Power Traders,...",M,...,"[I, I, M]","[NotFound, NotFound, 0]","[NonCore, NonCore, Core]","[M, I, M, M, F, M]","[0.628882614, NotFound, 1.137012322, 0.8983240...","[NonCore, NonCore, NonCore, NonCore, NonCore, ...",[],[],"[(14758, 1786)]",[1]
248,[{'content': 'We are continuing work to ensure...,[],28701,David Forster,We are continuing work to ensure our new onlin...,INITIAL,Online Trading Simulation,"[19927, 111470, 18656, 53481, 26954, 1621, 267...","[Tom Alonso, Tim Belden, Christopher F Calger,...",M,...,"[M, M, M, F, M, F, M, M, M, M, F, M, M, M, M, ...","[0.62200817, 0.48073235, 1.013790345, 0.465748...","[NonCore, NonCore, NonCore, NonCore, Core, Non...",[],[],[],"[(2273, 28701), (28701, 2619), (28701, 18989),...","[1, 1, 1, 1, 1, 1]",[],[]
277,[{'content': 'Email a Friend brought to you by...,[],836,Alan Comnes,Email a Friend brought to you by BayArea.com a...,INITIAL,CAISO Responds to McCullough's inane claims,"[2236, 19927, 7790, 2659, 2512, 111470, 26887,...","[Jim Buerkle, Tom Alonso, Alvarez, Ray, Robert...",M,...,"[M, M, M, M, M, M, M, M, M, M, M, M, M, I, F, ...","[0.684509435, 0.62200817, 0, 3.143279769, 0, 0...","[NonCore, NonCore, NonCore, NonCore, Core, Non...",[],[],[],"[(11290, 836)]",[1],[],[]
299,[{'content': 'With the filing of the Hart-Scot...,"[32737, 2619, 12499, 55055, 18989, 27723, 1939...",28701,David Forster,With the filing of the Hart-Scott-Rodino appli...,INITIAL,Online Launch Plan Update,"[32825, 32823, 32827, 32826, 14326]","[DL-NETCO Gas Traders, DL-NETCO Gas Originator...",M,...,"[I, I, I, I, M]","[NotFound, NotFound, NotFound, NotFound, 0]","[NonCore, NonCore, NonCore, NonCore, Core]","[I, F, F, F, M, M, I, M, M]","[NotFound, 0.755910493, 0.545606058, 0.5939341...","[NonCore, NonCore, NonCore, NonCore, NonCore, ...",[],[],"[(28701, 2619), (28701, 18989)]","[1, 1]"
314,[{'content': 'All: A more detailed version of...,"[1999, 7319, 28265, 35923]",836,Alan Comnes,All:\nA more detailed version of CAISO's MD02...,RE,California ISO Market Re-Design 2002 (MD02),"[836, 111470, 2512, 3493, 20403, 18397, 4758, ...","[Alan Comnes, Tim Belden, Robert Badeer, Chris...",M,...,"[M, M, M, M, M, M, M, M, M, M, M, M, M]","[0.497289084, 0.48073235, 0, 31.83223711, 31.8...","[NonCore, NonCore, Core, NonCore, NonCore, Non...","[F, F, M, F]","[0.304224573, 0.668081184, 0.418750947, 0.7637...","[NonCore, NonCore, NonCore, NonCore]",[],[],"[(1999, 836), (7319, 836)]","[1, 1]"
338,[{'content': 'To the West Desk et al: The FER...,[],836,Alan Comnes,To the West Desk et al:\n\nThe FERC's west-wid...,INITIAL,Details on the West-wide Price cap: now $108/M...,"[35923, 111470, 270, 7319, 1999, 35901, 106180...","[Sarah Novosel, Tim Belden, Richard Shapiro, J...",M,...,"[F, M, M, F, F, F, F, M, M, M, M, M, M, M, M, ...","[0.763758064, 0.48073235, 0, 0.668081184, 0.30...","[NonCore, NonCore, Core, NonCore, NonCore, Non...",[],[],[],"[(270, 836), (7319, 836), (1999, 836), (35901,...","[1, 1, 1, 1, 1]",[],[]
2701,[{'content': 'I have a very important v

In [223]:
print(data_power_only['main_text'][314])

All:
A more detailed version  of CAISO's MD02 proposal was released on Wednesday.  Below I am copying a summary prepared by Andy Brown who covers these matters for IEP.
It appears that the CAISO proposal will allow for SC's to go short or long in the R/T market (i.e., final schedules do not need to be balanced).  That's good.
I think the most material issue at this point is how the forward capacity requirement (ACAP, akin to ICAP) will affect our trading.
GAC
What is CAISO proposing?
First off, there are "target price" and "intra-zonal congestion management"
FERC filings that are apparently almost ready for a drop at FERC.  The
"comprehensive" package would build off that.
In essencence, CAISO is proposing a capacity market (similar to what IEP
proposed at the CPUC "Procurement OIR") that has an "available capacity
obligation" (ACAP) on the load-serving entities (LSEs, aka the utilities and
electric service providers).  This is the solution to the "all spot, all the
time" design that w